In [ ]:
# Neural Networks

# Class Notes:

# predict total number of bike rentals that occured?
# y is casual and registered combined.
# how do we want the data fed into the network?
# how will we use the data from 2020, the lockdown?
# combine values from hum and temp_c, feature enginere road conditions.
# trend from weather and covid.
# What were the exact dates the DC lockdown?
# frequency, average trend of busier times
# look at the tourist Season
# use both temp_c and temp_f

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
mini_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_day_of_year(row):
  date_list = row["dteday"].split("/")

  month = int(date_list[0])
  day = int(date_list[1])
  year = int(date_list[2]) # check for leap years

  if month == 1:
    days = day
  elif month == 2:
    days = day + 31
  elif month == 3:
    days = day + 31 + 28
  elif month == 4:
    days = day + 31 + 28 + 31
  elif month == 5:
    days = day + 31 + 28 + 31 + 30
  elif month == 6:
    days = day + 31 + 28 + 31 + 30 + 31
  elif month == 7:
    days = day + 31 + 28 + 31 + 30 + 31 + 30
  elif month == 8:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31
  elif month == 9:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31
  elif month == 10:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30
  elif month == 11:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31
  elif month == 12:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31 + 30

  if year % 4 == 0:
    days += 1

  return int(days)

def get_day_of_four_years(row):
  # there are 1461 days in four years but I want everything to be centered around election year
  year = int(row["dteday"].split("/")[2])
  day = int(row["dteday"].split("/")[1])

  days = ((year % 4) * 365) + day
  if year % 4 != 0:
    days += 1

  return int(days)

In [ ]:
def preprocess_bike_data(df):
    df["day_of_year"] = df.apply(get_day_of_year, axis=1)
    df["day_of_four_years"] = df.apply(get_day_of_four_years, axis=1)

    # Convert 'dteday' to datetime
    df['dteday'] = pd.to_datetime(df['dteday'], format='%m/%d/%Y')

    # Ensure 'hr' is an integer
    df['hr'] = df['hr'].astype(int)

    # Combine 'dteday' and 'hr' into a single 'datetime' column
    df['dteday'] = df.apply(lambda row: pd.Timestamp(row['dteday']) + pd.Timedelta(hours=row['hr']), axis=1)

    # Create year, month, and day columns
    df['year']        = df['dteday'].dt.year
    df['month']       = df['dteday'].dt.month
    df['day']         = df['dteday'].dt.day
    df['day_of_week'] = df['dteday'].dt.dayofweek

    # Define a function to convert Celsius to Fahrenheit
    def celsius_to_fahrenheit(celsius):
        return celsius * 9/5 + 32

    df['temp_f'] = df['temp_c'].apply(celsius_to_fahrenheit)
    df['feels_like_f'] = df['feels_like_c'].apply(celsius_to_fahrenheit)

    # Define the boundaries and labels for the temp_c bins
    temp_c_bins = [-14.7, -2.4, 9.2, 20.8, 40.5]
    temp_c_labels = ['very cold', 'cold', 'warm', 'hot']

    # Define the boundaries and labels for the feels_like_c bins
    feels_like_c_bins = [-24, -6.6, 3.6, 14.8, 49.6]
    feels_like_c_labels = ['very cold', 'cold', 'warm', 'hot']

    # Define the boundaries and labels for the windspeed bins
    windspeed_bins = [0, 17.45, 34.9, 52.35, 69.8]
    windspeed_labels = ['low', 'medium', 'high', 'very high']

    # Define the boundaries and labels for the hum bins
    hum_bins = [0.0889, 0.316725, 0.54455, 0.772275, 1.0]
    hum_labels = ['very low', 'low', 'medium', 'high']

    # Create new columns with the bin labels
    df['temp_c_bucket'] = pd.cut(df['temp_c'], bins=temp_c_bins, labels=temp_c_labels, include_lowest=True)
    df['feels_like_c_bucket'] = pd.cut(df['feels_like_c'], bins=feels_like_c_bins, labels=feels_like_c_labels, include_lowest=True)
    df['windspeed_bucket'] = pd.cut(df['windspeed'], bins=windspeed_bins, labels=windspeed_labels, include_lowest=True)
    df['hum_bucket'] = pd.cut(df['hum'], bins=hum_bins, labels=hum_labels, include_lowest=True)

    return df


In [ ]:
bikes = preprocess_bike_data(bikes)

bikes.head()

AttributeError: 'Timestamp' object has no attribute 'split'

In [ ]:
'''
This is the original testing split created by Makayla. This creates a randomized
train/ test split.
'''

'''
# Get dummies
bikes_dum = pd.get_dummies(bikes, columns=['hr', 'temp_c_bucket', 'feels_like_c_bucket', 'hum_bucket', 'windspeed_bucket', 'weathersit', 'season', 'holiday', 'workingday', 'year', 'month', 'day'])

# Split the data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Create the target variable
bikes_dum ['combined_y'] = bikes_dum ['casual'] + bikes_dum ['registered']
y = bikes_dum ['combined_y']

# features
# feature_columns = ['']
X = bikes_dum.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 17)

# Scale the data
minmax_scaler = preprocessing.MinMaxScaler()
X_train = minmax_scaler.fit_transform(X_train) # fit the scale to the training data
X_test = minmax_scaler.transform(X_test) # use the same scale on the testing data
'''

In [ ]:
'''
This is a chronological train/ test split.
'''

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Get dummies
bikes_dum = pd.get_dummies(bikes, columns=['hr', 'temp_c_bucket', 'feels_like_c_bucket', 'hum_bucket', 'windspeed_bucket', 'weathersit', 'season', 'holiday', 'workingday', 'year', 'month', 'day'])

# Ensure the data is sorted by 'datetime' to maintain chronological order
bikes_dum = bikes_dum.sort_values(by='dteday')

# Create the target variable
bikes_dum['combined_y'] = bikes_dum['casual'] + bikes_dum['registered']
y = bikes_dum['combined_y']

# Define indices for chronological splitting
split_index = int(len(bikes_dum) * 0.7)

# Split the data chronologically
# Get the size of bikes for train/validation/testing
train_size = int(0.7 * len(bikes))
val_size = int(0.15 * len(bikes))
test_size = len(bikes) - train_size - val_size

# Split the data for train/val/test
train_data = bikes_dum[:train_size]
val_data = bikes_dum[train_size:train_size + val_size]
test_data = bikes_dum[train_size + val_size:]

# Separate features and target for training and testing sets
y_train = train_data['combined_y']
y_test = test_data['combined_y']
y_val = val_data['combined_y']

X_train = train_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_test = test_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_val = val_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)

# Scale the data
minmax_scaler = preprocessing.MinMaxScaler()
X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)

# Display the shapes of the datasets to verify
print(f'Training data shape: {X_train.shape}, Training target shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing target shape: {y_test.shape}')
print(f'Validation data shape: {X_val.shape}, Validation target shape: {y_val.shape}')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Initialize the Neural Network
model = Sequential() # Sequential just means the network doesn't have loops--the outputs of one layer of neurons go to the next layer of neurons

# Add the first layer
model.add(Dense(16, input_dim=117, activation='relu')) # This layer has 16 neurons. They are each connected (dense) to the input neurons.
# Note: We want the input dimension to match the number of features at our input layer

# Add another "hidden layer"
model.add(Dense(8, activation = 'relu')) # This layer has 8 neurons

# Add the "output layer"
model.add(Dense(1, activation='linear')) # Our last layer doesn't need a non-linear activation function, unless it is useful for the type of answer we want
# The ouput layer should have the same number of neurons as outputs you are generating. In this case, it is just producing one number.

# Compile model
model.compile(loss='MSE', optimizer= 'Adam', metrics=['mean_squared_error'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, verbose=1)

Epoch 1/500
2461/2461 [==============================] - 7s 2ms/step - loss: 72767.3750 - mean_squared_error: 72767.3750 - val_loss: 48500.7539 - val_mean_squared_error: 48500.7539
Epoch 2/500
2461/2461 [==============================] - 9s 4ms/step - loss: 27885.7461 - mean_squared_error: 27885.7461 - val_loss: 48459.1836 - val_mean_squared_error: 48459.1836
Epoch 3/500
2461/2461 [==============================] - 6s 2ms/step - loss: 25780.4961 - mean_squared_error: 25780.4961 - val_loss: 47663.4453 - val_mean_squared_error: 47663.4453
Epoch 4/500
2461/2461 [==============================] - 7s 3ms/step - loss: 24611.2656 - mean_squared_error: 24611.2656 - val_loss: 48815.8125 - val_mean_squared_error: 48815.8125
Epoch 5/500
2461/2461 [==============================] - 6s 2ms/step - loss: 23439.0371 - mean_squared_error: 23439.0371 - val_loss: 48530.5234 - val_mean_squared_error: 48530.5234
Epoch 6/500
2461/2461 [==============================] - 8s 3ms/step - loss: 21460.1621 - mean_

In [ ]:
# Define the path to save the model
model_path = '/content/drive/My Drive/my_model.keras'

# Save the model
model.save(model_path)

In [ ]:
# 1. Check model architecture
print(model.summary())

# 2. Check the shape of input data
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
# Evaluate the model on the training data
_, train_mse = model.evaluate(X_train, y_train, verbose = 1)

# Evaluate the model on the testing data
_, test_mse = model.evaluate(X_test, y_test, verbose = 1)

In [ ]:
# Get predictions for the testing data
predictions = model.predict(X_test)

# Get the r^2
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print(r2)

In [ ]:
# Plot loss during training (we can do this because we saved a "history" during training)
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
# pyplot.ylim(0,20)

# Mini Holdout

In [ ]:
mini_holdout = test_data.drop(['dteday'], axis=1)
minmax_scaler = preprocessing.MinMaxScaler()
mini_holdout = minmax_scaler.fit_transform(mini_holdout)

mini_holdout_predictions = model.predict(mini_holdout)

#Format and Export data
predictions_table = pd.DataFrame(mini_holdout_predictions,columns=['predictions'])
predictions_table = predictions_table['predictions'].map({'yes': 1,'no': 0})
predictions_table.to_csv('team5-module4-predictions.csv',index=False)
predictions_table.value_counts()